# Link prediction - workshop maximum likelihood based methods

In this workshop we look at two methods for link prediction: **similarity based methods** and **maximum likelihood (ML) based methods**. The main idea behind similarity based methods, is that you choose a similarity metric (like the number of common neighbours) and base the reliability of an edge on the score from this metric. The art of this method is choosing a suitable similarity metric for your problem.

The idea behind ML methods is taking an underlying network model from which we assume the observed network was generated. Then, given the chosen model, we can compute the conditional probability that a certain edge is present in the graph given the observed network (using Bayes' rule). The higher this probability, the more likely it is that the link is missing. The art behind this method is choosing a suitable underlying random graph model your network is a realisation from. Sadly, ML methods are often difficult to implement. Therefore, we will look at one specific ML method in this workshop: one based on the **stochastic block model (SBM)**. 

**Exercise 1.** Give two reasons why the stochastic block model might be a good underlying model for real-life networks. Also, give two reasons why is might be poorly suited for real-life networks. 

## The likelihood function of the stochastic block model

The centrepiece of any ML method is the **likelihood function**. This function takes a network as input, and outputs the probability of this network appearing. For the stochastic block model, we know that each edge is present independently of the others with a probability that depends on the types of its adjacent nodes. If we know the stochastic block model's inputs $\vec{n}$ and $\textbf{P}$, then we can deduce that the likelihood function of a graph $G$ in this model is given by $$\mathbb{P}(G \;|\; \vec{n}, \textbf{P}) = \prod_{t \leq s} p_{ts}^{\ell_{ts}} (1 - p_{ts})^{r_{ts} - \ell_{ts}}.$$ Here, $\ell_{ts}$ is the number of links between vertex-types $t$ and $s$, and $r_{ts}$ is the maximum number of possible links between types $t$ and $s$. Finally, $p_{ts}$ is the entry at row $t$ and column $s$ of the matrix $\textbf{P}$, i.e. the probability that an edge between vertex-types $t$ and $s$ is present.

**Exercise 2.** Explain that the above likelihood function is correct. Then, implement it below. Test it on the generated stochastic block model underneath the function. *Hint: If implemented correctly, the likelihood should be approximately $6.25 \cdot 10^{-15}$.*

In [5]:
import networkx as nx
import numpy as np

def cLSBM(G, n, P):
    # Initialize likelihood
    likelihood = 1.0

    # Get the number of blocks
    k = len(n)
    
    # Get the node labels for each block
    node_blocks = {}
    start = 0
    for block_index in range(k):
        for node in range(start, start + n[block_index]):
            node_blocks[node] = block_index
        start += n[block_index]
    
    # Initialize block pair edge and possible edge counts
    ell = np.zeros((k, k))  # Number of edges between block pairs
    r = np.zeros((k, k))    # Maximum number of possible edges between block pairs
    
    # Calculate ell and r values
    for i in range(len(G.nodes)):
        for j in range(i + 1, len(G.nodes)):
            block_i = node_blocks[i]
            block_j = node_blocks[j]
            
            # Update the maximum number of possible edges r
            r[block_i][block_j] += 1
            if block_i != block_j:
                r[block_j][block_i] += 1
            
            # Update the number of observed edges ell
            if G.has_edge(i, j):
                ell[block_i][block_j] += 1
                if block_i != block_j:
                    ell[block_j][block_i] += 1

    # For intra-block possible edges (r values)
    for block in range(k):
        r[block][block] = (n[block] * (n[block] - 1)) / 2  # Combinations of 2 within the same block
    
    # Compute the likelihood
    for t in range(k):
        for s in range(t, k):
            p_ts = P[t][s]
            ell_ts = ell[t][s]
            r_ts = r[t][s]
            
            # Calculate likelihood component
            if r_ts > 0:  # To avoid log(0) issues
                likelihood *= (p_ts ** ell_ts) * ((1 - p_ts) ** (r_ts - ell_ts))
    
    return likelihood

# The code below tests your method

seed = 1 # Setting a random seed

# Generating the SBM
n = [5, 5, 5]
P = [[0.5, 0.1, 0.02], [0.1, 0.7, 0.1], [0.02, 0.1, 0.3]]
G = nx.stochastic_block_model(n, P, seed=seed)

# Computing likelihood
result = cLSBM(G, n, P)
print(result)


6.254395434954231e-15


From Exercise 2 you should notice that the likelihood for a relatively small network is already quite low. This is to be expected, since you are multiplying many numbers that are smaller than $1$. However, since the numbers are so small, this will quickly cause problems when networks become big: the likelihood function will always output the value $0$. Hence, we often need tricks to ensure the likelihood remains meaningful. The idea behind all such tricks is noting that we do not need to use the pure conditional probability $\mathbb{P}(G \;|\; \vec{n}, \textbf{P})$ as our reliability. 

**Exercise 3$\star$.** Find a way to adapt your answer to Exercise 2 that is also useful for larger graphs. Implement it in the code block below, and test it on a larger SBM.

In [6]:
import networkx as nx
import numpy as np

def cLogLSBM(G, n, P):
    # Initialize log-likelihood
    log_likelihood = 0.0

    # Get the number of blocks
    k = len(n)
    
    # Get the node labels for each block
    node_blocks = {}
    start = 0
    for block_index in range(k):
        for node in range(start, start + n[block_index]):
            node_blocks[node] = block_index
        start += n[block_index]
    
    # Initialize block pair edge and possible edge counts
    ell = np.zeros((k, k))  # Number of edges between block pairs
    r = np.zeros((k, k))    # Maximum number of possible edges between block pairs
    
    # Calculate ell and r values
    for i in range(len(G.nodes)):
        for j in range(i + 1, len(G.nodes)):
            block_i = node_blocks[i]
            block_j = node_blocks[j]
            
            # Update the maximum number of possible edges r
            r[block_i][block_j] += 1
            if block_i != block_j:
                r[block_j][block_i] += 1
            
            # Update the number of observed edges ell
            if G.has_edge(i, j):
                ell[block_i][block_j] += 1
                if block_i != block_j:
                    ell[block_j][block_i] += 1

    # For intra-block possible edges (r values)
    for block in range(k):
        r[block][block] = (n[block] * (n[block] - 1)) / 2  # Combinations of 2 within the same block
    
    # Compute the log-likelihood
    for t in range(k):
        for s in range(t, k):
            p_ts = P[t][s]
            ell_ts = ell[t][s]
            r_ts = r[t][s]
            
            # Calculate log-likelihood component
            if r_ts > 0:  # To avoid log(0) issues
                if p_ts > 0:
                    log_likelihood += ell_ts * np.log(p_ts)
                if p_ts < 1:
                    log_likelihood += (r_ts - ell_ts) * np.log(1 - p_ts)
    
    return log_likelihood

# The code below tests your method

seed = 42 # Setting a random seed

# Generating the SBM
n = [50, 50, 50]
P = [[0.5, 0.1, 0.02], [0.1, 0.7, 0.1], [0.02, 0.1, 0.3]]
G = nx.stochastic_block_model(n, P, seed=seed)

# Computing log-likelihood
result = cLogLSBM(G, n, P)
print(result)


-4247.370048761878


The likelihood function for the SBM we have now is relatively simple to use. Moreover, if $T_i$ is the type of vertex $i$ and $T_j$ the type of vertex $j$, then we also know that the likelihood of edge $\{i, j\}$ is given by entry $(T_i, T_j)$ in matrix $\textbf{P}$ (i.e., the value $p_{T_i, T_j}$). However, this analysis suffers from a major problem: it assumes we know the underlying SBM *exactly*. When we observe a network, however, we do not know the underlying SBM. All matrices $\textbf{P}$ and vectors $\vec{n}$ are fair game. Hence, to find the true likelihood, we need to integrate over all possible $\textbf{P}$ and $\vec{n}$.

Let us first see what happens if we do not know $\textbf{P}$, but do know $\vec{n}$ (and the exact allocation of types to vertices). In that case, if we observe a graph $G^O$ the reliability/likelihood will be given by $\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, G^O).$ Since we know the value of $\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, \text{P}, G^O) = p_{T_i T_j}$ we want to use the *law of total probability* to condition on the values in $\text{P}$. This yields the following integral: $$\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, G^O) = \iint_{[0, 1]^g} \mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, \textbf{P}, G^O) \cdot \mathbb{P}(\textbf{P} \;|\; \vec{n}, G^O) \; \text{d}\textbf{P}.$$
Here, $g$ is the number of distinct vertex-type pairs that are possible. Substituting what we know yields $$\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, G^O) = \iint_{[0, 1]^g} p_{T_i T_j} \cdot \mathbb{P}(\textbf{P} \;|\; \vec{n}, G^O) \; \text{d}\textbf{P}.$$

Now, to compute $\mathbb{P}(\textbf{P} \;|\; \vec{n}, G^O)$ we will use *Bayes' rule* to rewrite $$\mathbb{P}(\textbf{P} \;|\; \vec{n}, G^O) = \frac{\mathbb{P}(G^O \;|\; \vec{n}, \textbf{P} ) \mathbb{P}(\textbf{P}' \;|\; \vec{n})}{\iint_{[0, 1]^g} \mathbb{P}(G^O \;|\; \vec{n}, \textbf{P}' ) \mathbb{P}(\textbf{P}' \;|\; \vec{n}) \; \text{d}\textbf{P}'} = \frac{\prod_{t \leq s} p_{ts}^{\ell_{ts}} (1 - p_{ts})^{r_{ts} - \ell_{ts}} \mathbb{P}(\textbf{P} \;|\; \vec{n})}{\iint_{[0, 1]^g} \prod_{t \leq s} (p_{ts}')^{\ell_{ts}} (1 - (p_{ts}'))^{r_{ts} - \ell_{ts}} \mathbb{P}(\textbf{P}' \;|\; \vec{n}) \; \text{d}\textbf{P}'}.$$

Finally, we make the assumption that the values in the probability matrix $\textbf{P}'$ does not depend on the partitioning into groups $\vec{n}$. This means that $ \mathbb{P}(\textbf{P}' \;|\; \vec{n}) = c$ for some constant $c>0$ for all matrices $\textbf{P}'$. Hence, we find $$\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, G^O) = \iint_{[0, 1]^g} p_{T_i T_j} \cdot \frac{\prod_{t \leq s} p_{ts}^{\ell_{ts}} (1 - p_{ts})^{r_{ts} - \ell_{ts}} }{\iint_{[0, 1]^g} \prod_{t \leq s} (p_{ts}')^{\ell_{ts}} (1 - (p_{ts}'))^{r_{ts} - \ell_{ts}} \; \text{d}\textbf{P}'} \; \text{d}\textbf{P}.$$

Setting $C$ to be a (fixed) normalization constant, computing these integrals finally yields $$\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, G^O) = \frac{1}{C} \cdot \frac{\ell_{T_i T_j} + 1}{r_{T_i T_j} + 2} \prod_{t \leq s} \frac{1}{(r_{ts} + 1) \binom{r_{ts}}{\ell_{ts}}}.$$ 

**Exercise 4.** Look at the final expression after the derivaiton. Write down what is contained in the constant $C$, and explain why it is not needed to specify this number. Also, critically evaluate whether everything we do not "need" in the above expression is dumped into the constant $C$.

### Explanation of the Constant \( C \) in the Final Expression

To understand the final expression better, let's carefully analyze what the constant \( C \) represents and why it is not necessary to specify this number.

#### Final Expression Recap

The final expression we arrived at after performing the integration is:

$
\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, G^O) = \frac{1}{C} \cdot \frac{\ell_{T_i T_j} + 1}{r_{T_i T_j} + 2} \prod_{t \leq s} \frac{1}{(r_{ts} + 1) \binom{r_{ts}}{\ell_{ts}}}.
$


#### What Does \( C \) Contain?

The constant \( C \) is a normalization factor that depends on the integration over all possible matrices P. It encompasses all terms that are independent of the specific edge \( \{i, j\} \). To break it down:

$
C = \iint_{[0, 1]^g} \prod_{t \leq s} (p_{ts})^{\ell_{ts}} (1 - p_{ts})^{r_{ts} - \ell_{ts}} \; \text{d}\textbf{P}.
$

This integral represents the denominator in the application of Bayes' rule, where we integrate over all possible values of the matrix P. In other words, \( C \) ensures that the total probability distribution over all possible edges is properly normalized. It integrates over the likelihoods of all possible probability matrices P, given the observed graph \( G^O \) and the partition \( \vec{n} \).

#### Why Is \( C \) Not Needed to Specify?

\( C \) is not needed to specify because it is a **normalization constant** that does not affect the relative probabilities. In many applications of Bayesian inference and probabilistic modeling, it is common to work with expressions up to a proportional constant. Here's why:

1. **Relative Probabilities**: When calculating posterior distributions or likelihood ratios, the constant \( C \) cancels out. Thus, we often only need the probability up to a proportional constant to make decisions or comparisons.

2. **Integration Over All Possible States**: The integral represented by \( C \) involves all possible configurations of the probability matrix P. Computing this directly can be extremely difficult or infeasible in practice. However, for the purposes of inference or deriving relative likelihoods, we do not need the explicit value of C.

3. **Normalization in Probability**: Since C is a normalization factor, it guarantees that the sum (or integral) of all probabilities is 1. In many statistical models, normalization constants are considered implicitly rather than explicitly, as they do not affect the shape of the distribution.

4. **Irrelevance in Practical Computation**: In practical terms, especially for large models, explicitly calculating C is often avoided. Instead, algorithms (e.g., Markov Chain Monte Carlo, variational methods) work with unnormalized densities and focus on relative values.

#### Is Everything We Do Not "Need" Dumped into C?

Yes, everything that is not directly relevant to the calculation of the probability of a specific edge \( \{i, j\} \) given the observed graph \( G^O \) is absorbed into the constant C. This includes:

- The **normalization over all possible matrices** P, which accounts for the total space of possible configurations.
- Any terms that **do not depend on the specific edge** \( \{i, j\} \), but rather on the entire configuration of the graph.

The constant \( C \) is designed to capture all the integrals and summations that are computationally burdensome or irrelevant to the direct calculation of conditional probabilities. Thus, the approach simplifies the problem to manageable computations, leveraging the fact that we only care about ratios or relative magnitudes, not the exact probabilities.

### Conclusion

The constant C contains all terms needed to normalize the probability distribution over the entire space of possible configurations of P. It is not needed in practical computations since it does not affect relative probabilities or likelihoods. Everything that is not explicitly necessary for computing the desired conditional probability \( \mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, G^O) \) is indeed absorbed into C. This is a common technique in probabilistic modeling to focus on the essential computations without getting bogged down by intractable integrals.

**Exercise 5.** Load the network ``link_prediction_ML1.gz``. It is given that this network was generated through a stochastic block model where the first $50$ vertices are of type $1$, the next $30$ of type $2$ and the final $20$ of type $3$. Write code to perform link prediciton on this network using the SBM ML method given the above likelihood function. Given that we removed link $\{14, 48\}$, are the results as expected?

In [7]:
import networkx as nx
import numpy as np
import scipy.special as sp

# Load the network
G = nx.read_edgelist('/kaggle/input/network-statistics-for-data-science/link_prediction_ML1/link_prediction_ML1', nodetype=int)

# Define the partition sizes for the SBM
n = [50, 30, 20]  # Sizes of each group
type_labels = [1] * n[0] + [2] * n[1] + [3] * n[2]  # Type labels for nodes

# Initialize dictionaries to store observed and possible links between types
l_counts = np.zeros((3, 3))  # Observed links (ell_ts)
r_counts = np.zeros((3, 3))  # Possible links (r_ts)

# Compute the observed and possible link counts
for i in range(len(n)):
    for j in range(i, len(n)):
        # Get nodes of type i and j
        nodes_i = [node for node in G.nodes() if type_labels[node] == i + 1]
        nodes_j = [node for node in G.nodes() if type_labels[node] == j + 1]
        
        # Possible links (r_ts) between nodes of type i and j
        r_counts[i, j] = len(nodes_i) * len(nodes_j) if i != j else len(nodes_i) * (len(nodes_i) - 1) / 2
        
        # Observed links (ell_ts) between nodes of type i and j
        if i == j:
            l_counts[i, j] = len([edge for edge in G.edges(nodes_i) if type_labels[edge[0]] == i + 1 and type_labels[edge[1]] == i + 1]) / 2
        else:
            l_counts[i, j] = len([edge for edge in G.edges(nodes_i) if type_labels[edge[1]] == j + 1])

# Function to calculate the likelihood of a link existing
def compute_link_likelihood(l_counts, r_counts, i, j):
    Ti = type_labels[i] - 1  # Type of node i
    Tj = type_labels[j] - 1  # Type of node j
    ell_ts = l_counts[Ti, Tj]  # Observed links of types Ti, Tj
    r_ts = r_counts[Ti, Tj]  # Possible links of types Ti, Tj
    
    # Compute the probability for the link {i, j} using the likelihood formula
    prob = (ell_ts + 1) / (r_ts + 2)
    
    return prob

# Predict missing links using the likelihood method
missing_link_likelihoods = {}

for i in range(len(G.nodes())):
    for j in range(i + 1, len(G.nodes())):
        if not G.has_edge(i, j):  # Only consider missing links
            likelihood = compute_link_likelihood(l_counts, r_counts, i, j)
            missing_link_likelihoods[(i, j)] = likelihood

# Sort missing links by likelihood
sorted_links = sorted(missing_link_likelihoods.items(), key=lambda x: x[1], reverse=True)

# Check the likelihood of the removed link {14, 48}
removed_link = (14, 48)
removed_link_likelihood = compute_link_likelihood(l_counts, r_counts, removed_link[0], removed_link[1])

print(f"Likelihood of removed link {removed_link}: {removed_link_likelihood:.6f}")

# Check if the removed link is among the top predicted missing links
rank = next((rank for rank, link in enumerate(sorted_links) if link[0] == removed_link), None)

print(f"Rank of removed link {removed_link} in predicted missing links: {rank + 1 if rank is not None else 'Not in top predictions'}")


Likelihood of removed link (14, 48): 0.239609
Rank of removed link (14, 48) in predicted missing links: 328


Although the above derivation was already quite involved, you have probably noticed from Exercise 5 that giving away $\vec{n}$ and the exact location of each vertex-type is too much. There is not enough left to make the SBM ML link prediction method useful. Luckily (or to be fair sadly) we often also do not know (1) how many vertex-types there are, and (2) where these types are located. Thus, in reality the likelyhood will be given by $\mathbb{P}(\{i, j\} \in E \;|\; G^O)$. We can repeat the above caclulations to show that this will yields the following likelihood function for the stochastic block model: $$\mathbb{P}(\{i, j\} \in E \;|\; G^O) =  \frac{1}{C} \sum_{\textbf{T} \in \mathcal{T}} \frac{\ell_{T_i T_j} + 1}{r_{T_i T_j} + 2} \prod_{t \leq s} \frac{1}{(r_{ts} + 1) \binom{r_{ts}}{\ell_{ts}}}.$$ Here, the newly added sum forces us to compute the previous likelihood for all possible assignments of types to the vertices. So, $\mathcal{T}$ can be seen as the set of all type-assignments to the vertices in the graph, and $\textbf{T}$ can be seen as a signle sequence of vertex-types sampled from $\mathcal{T}$; we also call $\textbf{T}$ a **partition**. This is the likelihood function we need to make the SBM ML based method work. The question now becomes: how do we implement it?

**Exercise 6.** Look at the likelihood expression above, and answer the following questions. Write down your reasoning behind each answer.
- What is the value of $C$ in the likelihood expression above?
- Can the product in the likelihood expression above be taken out of the sum?
- Does the above likelihood function use global information about the graph $G^O$ or only local information around vertices $i$ and $j$?
- How many terms does the sum have? 
- How fast does the number of terms in the sum grow as $G^O$ has more vertices?

To analyze the likelihood expression provided in the context of stochastic block models (SBMs), let's answer each question step-by-step:

### Likelihood Function for Stochastic Block Model

The given likelihood function for predicting the existence of an edge \(\{i, j\}\) in a graph \(G^O\) is:

\[
\mathbb{P}(\{i, j\} \in E \;|\; G^O) =  \frac{1}{C} \sum_{\textbf{T} \in \mathcal{T}} \frac{\ell_{T_i T_j} + 1}{r_{T_i T_j} + 2} \prod_{t \leq s} \frac{1}{(r_{ts} + 1) \binom{r_{ts}}{\ell_{ts}}}.
\]

Here:
- \(\mathcal{T}\) is the set of all possible type-assignments (partitions) to the vertices in the graph.
- \(\textbf{T}\) is a single sequence of vertex types sampled from \(\mathcal{T}\).
- \(\ell_{T_i T_j}\) is the number of observed links between types \(T_i\) and \(T_j\).
- \(r_{T_i T_j}\) is the maximum possible number of links between types \(T_i\) and \(T_j\).

Now, let's analyze the questions:

### Question 1: What is the value of \(C\) in the likelihood expression above?

The constant \(C\) in the likelihood expression is a normalization constant. Its role is to ensure that the probability values computed from the likelihood function sum to 1 over all possible configurations. 

Given the sum over all possible type assignments \(\mathcal{T}\), the value of \(C\) can be expressed as:

\[
C = \sum_{\textbf{T} \in \mathcal{T}} \left( \frac{\ell_{T_i T_j} + 1}{r_{T_i T_j} + 2} \prod_{t \leq s} \frac{1}{(r_{ts} + 1) \binom{r_{ts}}{\ell_{ts}}} \right).
\]

This normalization constant \(C\) ensures that the total probability across all possible edges sums to 1. However, in the context of link prediction, knowing the exact value of \(C\) is generally not needed if we only care about the relative likelihood of different edges.

### Question 2: Can the product in the likelihood expression above be taken out of the sum?

No, the product in the likelihood expression cannot be taken out of the sum. This is because the product \(\prod_{t \leq s} \frac{1}{(r_{ts} + 1) \binom{r_{ts}}{\ell_{ts}}}\) depends on the specific partition \(\textbf{T}\) chosen from the set \(\mathcal{T}\). Each partition \(\textbf{T}\) gives a different set of \(\ell_{ts}\) values (observed links) and, consequently, different terms in the product. Therefore, the product must remain inside the sum.

### Question 3: Does the above likelihood function use global information about the graph \(G^O\) or only local information around vertices \(i\) and \(j\)?

The likelihood function uses **global information** about the graph \(G^O\). 

- The likelihood function sums over all possible type assignments \(\textbf{T}\) for the entire graph. This requires considering the overall structure of the graph and the distribution of links between all groups of nodes.
- The terms \(\ell_{ts}\) and \(r_{ts}\) represent the number of observed links and possible links between types \(t\) and \(s\) for the entire graph, not just around vertices \(i\) and \(j\). Thus, the likelihood function is inherently global.

### Question 4: How many terms does the sum have?

The number of terms in the sum corresponds to the number of possible type assignments \(\textbf{T}\) in the set \(\mathcal{T}\). If there are \(N\) vertices in the graph and \(K\) possible types (blocks), the number of terms in the sum is given by:

\[
|\mathcal{T}| = K^N.
\]

This is because each vertex can be assigned to any of the \(K\) types, leading to \(K^N\) possible type assignments.

### Question 5: How fast does the number of terms in the sum grow as \(G^O\) has more vertices?

The number of terms in the sum grows **exponentially** with the number of vertices \(N\). Specifically, it grows as \(K^N\), where \(K\) is the number of types (blocks). If \(K\) is fixed, the number of terms increases very rapidly as the number of vertices \(N\) increases. This exponential growth makes the computation intractable for large graphs, necessitating approximations or heuristic methods to efficiently estimate the likelihood.

### Conclusion

To summarize:

- \(C\) is a normalization constant that ensures the probabilities sum to 1.
- The product cannot be taken out of the sum because it depends on the specific partition.
- The likelihood function uses global information about the entire graph.
- The number of terms in the sum is \(K^N\).
- The number of terms grows exponentially with the number of vertices, making direct computation infeasible for large graphs.

## Approximating the likelihood function of the SBM

We now know what the likelihood function of the SBM maximum likelihood method for link prediction looks like, so now we only have to compute this likelihood and we have found the reliability of each vertex. However, as we have seen in Exercise 6, computing the likelihood function is cumbersome and computationally unfeasible if the number of vertices in $G^O$ grows. Thus, we need to find a way to approximate the likelihood function. Since there are too many options to choose from, a first idea we can try, is fixing the number of vertex-types present in the graph and choosing partitions of vertices into groups uniformly at random. You then compute the sum of likelihoods over only those vertex types assignments you have sampled. 

In the rest of this workshop we will consider the (real-life) dataset in ``link_prediction_ML2.gz``. This dataset depicts books about US politics published around the 2004 election. An edge between two books means that the books have often been purchased together on "amazon.com". We have removed edge $\{40, 44\}$. 

**Exercise 7$\star$.** Think about the nature of the network ``link_prediction_ML2.gz``. How many vertex-types do you think there are? Use this knowledge to approximate the likelihood function, and do link prediciton using the SBM ML method on the dataset. How well does the method perform?

*Hint: You can use the skeleton below to guide you*

In [9]:
import networkx as nx
import numpy as np
import math
import random

def twoClassSBMLinkPrediction(Adj, trials=100, noTypes=2):
    """
    Performs SBM link prediction by randomly sampling partitions with a given number of types.
    
    Parameters
    ----------
    Adj : Symmetric np.array() with {0, 1} entries;
        Adjacency matrix of the input graph.
    trials : int, optional;
        The number of partitions we sample. The default is 100.
    noTypes : int, optional;
        The number of vertex-types we assume. The default is 2.
    
    Returns
    -------
    reliability : np.array() with float entries;
        A matrix with the reliabilities of all edges.  
    """
    n = Adj.shape[0]  # Number of vertices
    pReliability = np.zeros((trials, n, n))  # To store edge reliabilities over all trials

    for trial in range(trials):
        # Step 1: Sample a random partition (i.e., assign vertices to one of the noTypes groups)
        # Randomly assign each vertex to a type (community)
        partition = np.random.randint(0, noTypes, size=n)

        # Step 2: Compute the number of vertices in each type
        type_counts = np.zeros(noTypes)
        for t in range(noTypes):
            type_counts[t] = np.sum(partition == t)

        # Initialize block-wise probabilities (edge existence between types)
        block_probabilities = np.zeros((noTypes, noTypes))

        # Step 3: Compute edge-dependent part of the likelihood function for each edge
        for i in range(n):
            for j in range(i + 1, n):
                type_i = partition[i]
                type_j = partition[j]

                # Count edges between types to estimate block probabilities
                if Adj[i, j] == 1:
                    block_probabilities[type_i, type_j] += 1
                    if type_i != type_j:
                        block_probabilities[type_j, type_i] += 1  # Symmetric case

        # Normalize block probabilities (divide by the number of possible edges between types)
        for t1 in range(noTypes):
            for t2 in range(noTypes):
                if t1 == t2:
                    possible_edges = type_counts[t1] * (type_counts[t1] - 1) / 2
                else:
                    possible_edges = type_counts[t1] * type_counts[t2]

                if possible_edges > 0:
                    block_probabilities[t1, t2] /= possible_edges

        # Step 4: Use the block probabilities to predict edges (reliability)
        for i in range(n):
            for j in range(i + 1, n):
                type_i = partition[i]
                type_j = partition[j]
                edge_prob = block_probabilities[type_i, type_j]

                # Store the reliability of this edge in the current trial
                pReliability[trial, i, j] = edge_prob
                pReliability[trial, j, i] = edge_prob  # Symmetry

    # Step 5: Sum over all computed reliabilities across partitions
    reliability = np.mean(pReliability, axis=0)  # Average over all trials

    return reliability

# Load the edgelist into a NetworkX graph
Adj = "/kaggle/input/network-statistics-for-data-science/link_prediction_ML2/link_prediction_ML2"
G_adj = nx.read_edgelist(Adj, nodetype=int)

# Convert the graph to a NumPy adjacency matrix
Adj_matrix = nx.to_numpy_array(G_adj)

# Perform SBM link prediction
reliability_matrix = twoClassSBMLinkPrediction(Adj_matrix)

# Output the reliability matrix
print(reliability_matrix)


[[0.         0.07968943 0.07941604 ... 0.08108977 0.07964128 0.08018139]
 [0.07968943 0.         0.079454   ... 0.08015862 0.07967    0.07976239]
 [0.07941604 0.079454   0.         ... 0.08018828 0.08051544 0.07965716]
 ...
 [0.08108977 0.08015862 0.08018828 ... 0.         0.0805395  0.07976555]
 [0.07964128 0.07967    0.08051544 ... 0.0805395  0.         0.07998468]
 [0.08018139 0.07976239 0.07965716 ... 0.07976555 0.07998468 0.        ]]


On of the problems you might have encountered in Exercise 7, is that the term $$f(\textbf{T}) = \prod_{t \leq s} \frac{1}{(r_{ts} + 1) \binom{r_{ts}}{\ell_{ts}}}$$ in the likelihood function is quite dominating. Hence, it is not very useful to saple partitions uniformly at random, since many of them will not contribute much to the final result anyways. We need to find a way to sample *relevant* partitions only. To do this, we apply the following idea:
1. Start with a random partition $\textbf{T}_{\text{old}}$.
2. Change the type of one of the vertices in $\textbf{T}_{\text{old}}$ randomly to create $\textbf{T}_{\text{new}}$.
3. Compute $f(\textbf{T}_{\text{new}})$. We accept the change in Step 2 (and set $\textbf{T}_{\text{old}}$ to be equal to $\textbf{T}_{\text{new}}$) if:
    * The value $f(\textbf{T}_{\text{new}})$ is larger than $f(\textbf{T}_{\text{old}})$, or
    * If $f(\textbf{T}_{\text{new}})$ is smaller than $f(\textbf{T}_{\text{old}})$, then accept the change only with probability $f(\textbf{T}_{\text{new}}) / f(\textbf{T}_{\text{old}})$.
4. After a large enough number of accepted changes, the value of $f(\textbf{T}_{\text{new}})$ should have converged to the value "important" partitions take. Now, we can start to sample partitions, and compute the likelihoods of them.
5. Repteat Steps 2 and 3 above until a fixed number of partition changes have been accepted.
6. Compute the likelihood of the partition you have obtained after Step 5 for all edges and store it.
7. Reteat Steps 5 and 6 until you have enough samples of the likelihood function. Sum over all obtained partition to find the reliability of each edge.

Another problem you might have faced in Exercise 7, is that the value $C$ was hard to guess and that the likelihood function was numerically unstable. There are some practicalities to keep in mind with regards to these issues:
* Working with $f(\textbf{T}_{\text{new}})$ directly is numerically unstable if you have many possible vertex-types. You might want to consider using only $\log(f(\textbf{T}_{\text{new}}))$.
* You need to choose the normalization constant $C$ in the likelihood function cleverly. It might be smart to save the "converged" value of $f(\textbf{T}_{\text{old}})$ and use that is your value of $C$, since you expect all subsequent evaluations of $f$ to be close to this value.

**Exercise 8.** Implement the above idea to do link prediction with the likelihood based stochastic block model method on ``link_prediction_ML2.gz``. How does it perform? To help you get started, below you will find a function that executes Step 1 through 4. Check this function thoroughly, so you understand what it does; there might still be tiny mistakes in it.

*Hint: You may fix the number of vertex-types, but you may also keep it unknown. The idea of the algorithm works in both settings, but the implementation is slightly simpler if you know the number of vertex-types.*

In [12]:
import networkx as nx
import numpy as np
import math
import random


def initialConfigF(Adj, noTypes=100, trainSteps=1000):
    """
    Performs initial convergence of f(P) in the Metropolis algorithm.

    Parameters
    ----------
    Adj : Symmetric np.array() with {0, 1} entries;
        Adjacency matrix of the input graph.
    noTypes : int, optional;
        The number of communities we assume. The default is 100.
    trainSteps : int, optional;
        The number of iterative steps taken to converge f(P). The default is 1000.

    Returns
    -------
    Fval : float;
        The final (converged) value of log(f(P)). The logarithm is given to
        keep the algorithm numerically stable.
    oldP : np.array() with int entries;
        The partition P that produced f(P).
    """
    # Initial training for f
    Fval = - 10**10
    oldP = np.random.choice(noTypes, Adj.shape[0])  # Choose a random partition

    for step in range(trainSteps):
        fail = True

        # Keep trying new partition until you've found one that is accepted
        while fail:
            # Create new P
            index = np.random.randint(Adj.shape[0])
            newP = oldP.copy()
            newP[index] = np.random.choice(np.delete(np.arange(noTypes), oldP[index]))

            # Compute the logarithm of f(P) [for numerical stability]
            n = np.histogram(newP, bins=np.arange(noTypes + 1))[0]
            normconst = 0
            for k, val1 in enumerate(n):
                for l, val2 in enumerate(n[k:]):
                    l = k + l  # Shift the index l to the correct value
                    rDummy = val1 * val2
                    c = Adj[newP == k, :]
                    d = c[:, newP == l]
                    ellDummy = np.sum(d)
                    if k == l:
                        rDummy = (val1 - 1) * val1 / 2
                        ellDummy = ellDummy / 2
                    if rDummy <= 50 or ellDummy == 0:
                        normconst += -np.log(rDummy + 1) - np.log(math.comb(int(rDummy), int(ellDummy)))
                    else:
                        # Approximation of logarithm of binomial coefficient
                        normconst += -np.log(rDummy + 1) - (rDummy * np.log(rDummy) - ellDummy * np.log(ellDummy) - (rDummy - ellDummy) * np.log(rDummy - ellDummy))

            # Test whether we accept the change, and if accepted: update
            if np.random.binomial(1, min([1., np.exp(normconst - Fval)])) == 1:
                Fval = normconst
                oldP = newP.copy()
                fail = False

    return Fval, oldP


def sbmLinkPrediction(Adj, noTypes=100, trainSteps=1000, testSet=None):
    """
    Performs link prediction using SBM and returns the predicted probabilities.

    Parameters
    ----------
    Adj : np.array
        Adjacency matrix of the graph.
    noTypes : int
        The number of community types.
    trainSteps : int
        The number of steps for partition convergence.
    testSet : list of tuples, optional
        List of node pairs in the test set to predict links for.

    Returns
    -------
    reliability : np.array
        Reliability matrix containing link prediction scores.
    """
    # Get initial partition using the Metropolis algorithm
    Fval, partition = initialConfigF(Adj, noTypes=noTypes, trainSteps=trainSteps)

    # Create block probability matrix
    n = len(partition)
    block_probabilities = np.zeros((noTypes, noTypes))
    type_counts = np.zeros(noTypes)

    # Count the number of edges between each pair of communities
    for i in range(n):
        for j in range(i + 1, n):
            type_i = partition[i]
            type_j = partition[j]
            if Adj[i, j] == 1:
                block_probabilities[type_i, type_j] += 1
                if type_i != type_j:
                    block_probabilities[type_j, type_i] += 1

    # Normalize block probabilities based on number of possible edges
    for i in range(noTypes):
        for j in range(noTypes):
            if i == j:
                possible_edges = type_counts[i] * (type_counts[i] - 1) / 2
            else:
                possible_edges = type_counts[i] * type_counts[j]
            if possible_edges > 0:
                block_probabilities[i, j] /= possible_edges

    # Predict links for the test set based on block probabilities
    reliability = np.zeros((n, n))
    for i in range(n):
        for j in range(i + 1, n):
            type_i = partition[i]
            type_j = partition[j]
            reliability[i, j] = block_probabilities[type_i, type_j]
            reliability[j, i] = reliability[i, j]

    return reliability


# Example usage:
# Load the adjacency matrix from the gzipped file
#Adj = np.loadtxt("link_prediction_ML2.gz")
Adj = "/kaggle/input/network-statistics-for-data-science/link_prediction_ML2/link_prediction_ML2"
G_adj = nx.read_edgelist(Adj, nodetype=int)
Adj_matrix = nx.to_numpy_array(G_adj)

# # Perform SBM link prediction
# reliability_matrix = sbmLinkPrediction(Adj_matrix)
# print(reliability_matrix)



## Validation

In certain exercises you were asked to assess the performance of a link prediction algorithm. You have probably found out that this is not an easy task. In general, validation for link prediction is a difficult talk, since we do not know that the actual network looks like. Usually this lack of a ground truth is mediated by using **cross-validation** methods. The idea is to remove extra edges from a network, and using the link prediction method to assess how well these extra edges are retrieved. By doing this multiple times, and averaging over the results you can obtain a score that tells you how well the link prediction method performs on a given problem.

**Exercise 9.** The code below implements a specific cross-validation method for the Jaccard index (this is a similarity based link prediction method). Find out what the code does, and add comments to explain the procedure. Then, adapt it to the SBM maximum likelihood link prediction method, and use it to test the performance of this algorithm.

In [ ]:
def leaveOneOutJaccard(G):
    AUC = 0
    for e in G.edges:
        Gprobed = G.copy()
        Gprobed.remove_edge(e[0],e[1])
        target = nx.jaccard_coefficient(Gprobed, [e])
        for _, _, p in target:
            reliabilityTarget = p
        
        reliability = nx.jaccard_coefficient(Gprobed)
        better = 0
        same = 0
        total = 0
        for _, _, p in reliability:
            if p < reliabilityTarget:
                better += 1
            elif p == reliabilityTarget:
                same += 1
            total += 1
        
        AUC += (0.5 * same + better) / total
    return AUC / len(G.edges)


def leaveOneOutSBM(Adj, G, noTypes=2, trainSteps=1000):
    """
    Leave-One-Out Cross-Validation for SBM Maximum Likelihood Link Prediction

    Parameters:
    -----------
    Adj : np.array
        Adjacency matrix of the graph.
    G : networkx.Graph
        The graph to test on.
    noTypes : int
        Number of communities for SBM.
    trainSteps : int
        Number of steps for the Metropolis algorithm in SBM.
    
    Returns:
    --------
    AUC : float
        Area Under the Curve (AUC) score for SBM link prediction.
    """
    AUC = 0  # Initialize AUC score to 0
    
    # Loop over all edges in the graph for leave-one-out cross-validation
    for e in G.edges:
        # Create a probed version of the adjacency matrix with the edge removed
        Gprobed = G.copy()
        Gprobed.remove_edge(e[0], e[1])
        
        # Get the new adjacency matrix after the edge removal
        Adj_probed = nx.to_numpy_array(Gprobed)
        
        # Perform SBM link prediction on the modified graph
        reliability_matrix = sbmLinkPrediction(Adj_probed, noTypes=noTypes, trainSteps=trainSteps)
        
        # Store the reliability score for the target (removed) edge
        reliabilityTarget = reliability_matrix[e[0], e[1]]
        
        # Compare the reliability score of all edges to the target edge
        better = 0
        same = 0
        total = 0
        
        # Iterate over all node pairs and compare their reliability scores to the target edge
        for i in range(Adj.shape[0]):
            for j in range(i + 1, Adj.shape[0]):
                if Gprobed.has_edge(i, j) or (i, j) == e:
                    continue  # Skip pairs that are already connected, and skip the removed edge
                reliability = reliability_matrix[i, j]
                if reliability < reliabilityTarget:
                    better += 1  # Count if the score is lower than the target score
                elif reliability == reliabilityTarget:
                    same += 1  # Count if the score is equal to the target score
                total += 1  # Count total comparisons
        
        # Update the AUC based on comparisons made
        AUC += (0.5 * same + better) / total  # Weight same scores as 0.5, better scores as 1
    
    # Return the average AUC score over all edges
    return AUC / len(G.edges)


# Load the adjacency matrix and create a graph
Adj = "/kaggle/input/network-statistics-for-data-science/link_prediction_ML2/link_prediction_ML2"
G_adj = nx.read_edgelist(Adj, nodetype=int)
Adj_matrix = nx.to_numpy_array(G_adj)

# Perform leave-one-out cross-validation for SBM link prediction
AUC_SBM = leaveOneOutSBM(Adj_matrix, G_adj, noTypes=3, trainSteps=1000)

print(f"AUC Score for SBM Link Prediction: {AUC_SBM}")


/tmp/ipykernel_36/4172621528.py:62: RuntimeWarning: overflow encountered in exp
  if np.random.binomial(1, min([1., np.exp(normconst - Fval)])) == 1:


In this workshop you have now tried the stochastic block model maximum likelihood method for link prediction on both real and synthetic data. You should now have noticed the following:
- Link prediction is a difficult task, and in practice it is almost impossible to retrieve the missing links exactly.
- Maximum likelihood methods are often difficult to implement and by the nature of the likelihood function numerically unstable.
- The performance of maximum likelihood methods heavily depends on how suitible your underlying null model is for the problem.
- You need to understand the stochastic structure of the underlying null model well if you want to use it for link predictoin.
- It is not easy to validate and assess performance of link prediction algorithms (more on this in a later lecture).